# Data Encoding, Decoding and Flow

## Protocol Buffers (Protobuf)

Protobuf types include:

- double: double precision floating point number
- float: single precision floating point number
- int32: signed integer, uses variable-length encoding
- int64: signed integer, uses variable-length encoding
- uint32: unsigned integer, uses variable-length encoding
- uint64: unsigned integer, uses variable-length encoding
- sint32: signed integer, uses variable-length encoding, more efficient than int32
- sint64: signed integer, uses variable-length encoding, more efficient than int64
- fixed32: unsigned integer, always 4 bytes
- fixed64: unsigned integer, always 8 bytes
- sfixed32: signed integer, always 4 bytes
- sfixed64: signed integer, always 8 bytes
- bool: boolean value
- string: UTF-8 text string
- bytes: sequence of bytes
- enum: enumerated type
- message: nested message type
- map: map type
- Any: dynamic type

Protobuf schema definitions are defined in `.proto` files. The Protobuf compiler generates code in various languages from the `.proto` files.

### Encoding

We can encode the previous example record in Protobuf using the following schema in the `.proto` file:

```protobuf
message Person {
  required string user_name = 1;
  optional int64 favorite_number = 2;
  repeated string interests = 3;
}
```

The data encoded with this schema looks like this:
![protobuf](../assets/protobuf.png)

Protocol Buffers have an interesting aspect regarding its datatype handling. Unlike having a specific list or array datatype, it utilizes a `repeated` marker for fields, which serves as a third option alongside `required` and `optional`.

As depicted in the figure, a repeated field is simply represented by the same field tag appearing multiple times in the record. The advantage of this approach is that converting an optional (single-valued) field into a repeated (multi-valued) field is permissible. When new code reads old data, it interprets it as a list with either zero or one element, depending on whether the field was present. On the other hand, old code reading new data only perceives the last element of the list.

### gRPC

Nows, let's look at how to use the generated code to make remote procedure calls. We will use gRPC, which is a high-performance RPC framework built on top of Protobuf. gRPC is a client-server application where the client initiates an RPC call and waits for a response from the server. The server executes the requested operation and returns a response to the client.

In [1]:
%%writefile ../schema/person.proto
syntax = "proto3";

message Person {
  string user_name = 1;
  optional int64 favorite_number = 2;
  repeated string interests = 3;
}

message CourseRequest {
  Person person = 1;
  string course = 2;
}


service School {
  rpc teachCourse(CourseRequest) returns (Person) {}
}

Overwriting ../schema/person.proto


Then, run the following command in a terminal to generate the Python code:

```bash
python -m grpc_tools.protoc -I./schema --python_out=. --grpc_python_out=. ./schema/person.proto
```

This will generate the following files:

```bash
person_pb2.py
person_pb2_grpc.py
```

In [2]:
%%writefile ../person_protobuf_server.py
from concurrent import futures
import grpc
import person_pb2_grpc


class School(person_pb2_grpc.SchoolServicer):

  def teachCourse(self, request, context):
    request.person.interests.append(request.course)
    return request.person

server = grpc.server(futures.ThreadPoolExecutor(max_workers=2))
person_pb2_grpc.add_SchoolServicer_to_server(
    School(), server)
server.add_insecure_port('[::]:50051')
server.start()
server.wait_for_termination()

Overwriting ../person_protobuf_server.py


Then, run `python person_protobuf_server.py` in a new terminal. This will start the server.

In [3]:
import sys
sys.path.append('..')
import grpc
import person_pb2
import person_pb2_grpc


In [4]:

with grpc.insecure_channel('localhost:50051') as channel:
    martin = person_pb2.Person(user_name='Martin', favorite_number=1337, interests=["daydreaming", "hacking"])
    course = "coding"
    stub = person_pb2_grpc.SchoolStub(channel)
    updated_martin = stub.teachCourse(person_pb2.CourseRequest(person=martin, course=course))
    print(updated_martin.interests)

['daydreaming', 'hacking', 'coding']


> Add a new field `grade` (0-100) with an appropriate type annotation to the `Person` message. Then, add a new method `assignGrade` to the `School` service that takes a `GradeRequest` message (which consists of a `Person` record and a `grade` arguments), assigns the `grade` to the `Person` and returns the `Person`. Then call the method by passing "Martin" `person` and a grade number, and print his grade.

In [5]:
%%writefile ../schema/person.proto

syntax = "proto3";

message Person {
  string user_name = 1;
  optional int64 favorite_number = 2;
  repeated string interests = 3;
  optional int32 grade = 4;  // Grade field (0-100)
}

message CourseRequest {
  Person person = 1;
  string course = 2;
}

message GradeRequest {
  Person person = 1;
  int32 grade = 2;  // Grade value (0-100)
}

service School {
  rpc teachCourse(CourseRequest) returns (Person) {}
  rpc assignGrade(GradeRequest) returns (Person) {}
}

Overwriting ../schema/person.proto


Then, rerun the following command in a terminal to generate the updated python files for person.proto:

```bash
python -m grpc_tools.protoc -I./schema --python_out=. --grpc_python_out=. ./schema/person.proto
```

This will generate the following files:

```bash
person_pb2.py
person_pb2_grpc.py
```

In [6]:
%%writefile ../person_protobuf_server.py

from concurrent import futures
import grpc
import person_pb2_grpc


class School(person_pb2_grpc.SchoolServicer):

    def teachCourse(self, request, context):
        request.person.interests.append(request.course)
        return request.person
    
    def assignGrade(self, request, context):
        # Assign the grade from the request to the person
        request.person.grade = request.grade
        return request.person

server = grpc.server(futures.ThreadPoolExecutor(max_workers=2))
person_pb2_grpc.add_SchoolServicer_to_server(
    School(), server)
server.add_insecure_port('[::]:50051')
server.start()
server.wait_for_termination()

Overwriting ../person_protobuf_server.py


Then, run `python person_protobuf_server.py` in a new terminal. This will start the server.

NOTE: this requires restarting the kernel, so cached values of the server will be replaced with new version with assignGrade method

In [1]:
import sys
sys.path.append('..')
import grpc
import person_pb2
import person_pb2_grpc

In [5]:
with grpc.insecure_channel('localhost:50051') as channel:
    martin = person_pb2.Person(user_name='Martin', favorite_number=1337, interests=["daydreaming", "hacking"])
    course = "data science"
    stub = person_pb2_grpc.SchoolStub(channel)
    
    # First, teach the course to add "coding" to interests
    updated_martin = stub.teachCourse(person_pb2.CourseRequest(person=martin, course=course))
    
    # Then, assign a grade to the same person
    final_martin = stub.assignGrade(person_pb2.GradeRequest(person=updated_martin, grade=90))
    
    # Display both interests and grade
    print(f"Martin's interests: {final_martin.interests}")
    print(f"Martin's grade: {final_martin.grade}")

Martin's interests: ['daydreaming', 'hacking', 'data science']
Martin's grade: 90
